<a href="https://colab.research.google.com/github/vaquejano/robo_V1UEM/blob/main/controle%20Robo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 as cv
import numpy as np #biblioteca para contas matematicas
import numpy as np #biblioteca para contas matematicas
import time
import RPi.GPIO as GPIO
from gpiozero.pins.pigpio import PiGPIOFactory
import math

#####motores
GPIO.setwarnings(False)
GPIO.setmode(GPIO.BOARD)
RF = 12 #Right - Forward
RB = 32 #Right - Backward
LF = 33 #Left  - Forward
LB = 35 #Left  - Backward
GPIO.setup(RF, GPIO.OUT) 
GPIO.setup(RB, GPIO.OUT)
GPIO.setup(LF, GPIO.OUT)
GPIO.setup(LB, GPIO.OUT)
motor_RF = GPIO.PWM(RF, 100)
motor_RB = GPIO.PWM(RB, 100)
motor_LF = GPIO.PWM(LF, 100)
motor_LB = GPIO.PWM(LB, 100)
motor_RF.start(0)
motor_RB.start(0)
motor_LF.start(0)
motor_LB.start(0)
#####motores


#captura = cv.VideoCapture('03.mp4')
captura = cv.VideoCapture(0)
captura.set(cv.CAP_PROP_FRAME_WIDTH,  480)
captura.set(cv.CAP_PROP_FRAME_HEIGHT, 320)


frame_width = int(captura.get(3)) 
frame_height = int(captura.get(4)) 
size = (frame_width, frame_height) 

#################################################### VARIAVEIS
meio_amarelo = 0
stop = 1
velocidade = 0
KP = 0.1
KI = 0
KD = 0
i_erro = 0
erro_anterior = 0
d = int(input("Digite um valor para tolerancia:"))

'''
video_cod = cv.VideoWriter_fourcc(*'XVID')
video_output= cv.VideoWriter('captured_video.avi',
                      video_cod,
                      10,
                      (frame_width,frame_height))
'''
                     
while(1):
	_, frame = captura.read()
	img_cv = cv.rotate(frame, cv.ROTATE_180)  #rotaciona o video
	#cv.imshow("Video", frame) 
	#cv.imshow("Video", frame) 
	#preparação da imagem
	mask = np.zeros_like(img_cv)
	h = 320	
	w = 480
	#Gerar a Mascara - faz o corte na imagem
	pts = np.array([[0,480],[0,150],[680,150],[680,480]], dtype=np.int32) # P1, P2, P3, P4
	mask_fillpoly = mask #preencher o poligono
	masked_filled = cv.fillPoly(mask_fillpoly, [pts], (255, 255, 255)) #pega a mascara, os pontos e preenche de amarelo
	masked_image = cv.bitwise_and(img_cv, masked_filled) #bitwise no trapezio 
	img_blur = cv.blur(masked_image, (5, 5))
	#converter a cor para hsv para detectar o amarelo
	img_hsv = cv.cvtColor(masked_image, cv.COLOR_BGR2HSV) 
	#converter para rgb para detectar o branco
	img_rgb = cv.cvtColor(masked_image, cv.COLOR_BGR2RGB) 
	#detecta amarelo
	img_amarelo = img_hsv
	low = np.array([25, 90, 90])
	high = np.array([36, 255, 255])
	mask = cv.inRange(img_amarelo, low, high)
	res = cv.bitwise_and(img_amarelo,img_amarelo, mask= mask)
	contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
	contours = sorted(contours, key=lambda x:cv.contourArea(x), reverse=True)
	menor_x = 600
	menor_y = 0
	maior_x = 0
	maior_y = 400
	x0_amarelo = 0
	x1_amarelo = 0
	y0_amarelo = 0
	y1_amarelo = 0
	x0_branco = 0
	y0_branco = 0
	x1_branco = 0
	y1_branco = 0
	meio_branco = 0
	#detecta amarelo
	for cnt in contours:
	  (x, y, w, h) = cv.boundingRect(cnt)
	  cv.rectangle(img_cv, (x, y+h), (x+w, y), (255,0, 0), 2)
	  #cv.line(img_cv,(x,y+h),(x+w, y), (0,0,255), 2)
	  x0_amarelo = x
	  y0_amarelo = y
	  x1_amarelo = x + w
	  y1_amarelo = y + h
	  meio_amarelo = int(x + w/2)
	  #desenha retangulo do amarelo na tela
	 # if meio_amarelo > 300:
	  #	meio_amarelo = 300
	  cv.line(img_cv, (meio_amarelo,0), (meio_amarelo, 320), (255,0,0), 2)
	  break
	
	#cv.line(img_cv, (menor_x,menor_y), (maior_x, maior_y), (255,0,0), 2)  
		  
	#blur na imagem rgb
	img_branca = cv.blur(img_rgb, (15, 15))

	#detecta branco
	low = np.array([130, 130, 130])
	high = np.array([240, 240, 240])
	mask = cv.inRange(img_branca, low, high)
	res = cv.bitwise_and(img_branca,img_branca, mask= mask)
	contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
	contours = sorted(contours, key=lambda x:cv.contourArea(x), reverse=True)
	for cnt in contours:
	  (x, y, w, h) = cv.boundingRect(cnt)
	  cv.rectangle(img_cv, (x, y), (x+w, y+h), (0,255, 0), 2)
	  #cv.line(img_cv,(x,y),(x+w, y+h), (0,0,255), 2)
	  x0_branco = x
	  y0_branco = y
	  x1_branco = x + w
	  y1_branco = y + h
	  meio_branco = int(x + w/2)
	  cv.line(img_cv, (meio_branco,0), (meio_branco, 320), (255,0,0), 2)
	  break
	#plt.imshow(img_cv)

	"""**matematica da coisa**"""

	#parametros da reta amarela:
	a = (x1_amarelo -x0_amarelo )
	if a == 0:
		a = 1
	m_amarelo = (y1_amarelo - y0_amarelo)/a
	if m_amarelo >= 0:
		m_amarelo = -0.1
	n_amarelo = y1_amarelo -(m_amarelo*x1_amarelo)
	#ponto pertencente a reta amarela
	x3 = (240-n_amarelo)/m_amarelo
	distancia_amarelo = int(320 - meio_amarelo)
	#parametros da reta branco:
	m_branco =1# (y1_branco - y0_branco)/(x1_branco -x0_branco)
	n_branco = y1_branco -(m_branco*x1_branco)
	#ponto pertencente a reta branca
	x_do_branco = (240-n_branco)/m_branco
	distancia_branco = int(meio_branco - 240)
	
	#parte grafica da reta de distancia do amarelo
	if meio_amarelo > 240:
		meio_amarelo = 240
	cv.line(img_cv,(meio_amarelo,160),(240,160), (0,255,255),2) 
	
	if distancia_amarelo < 0:
		distancia_amarelo = -480
	texto = ('dist. amarelo = ' + str(int(distancia_amarelo)))
	cv.putText(img_cv, texto,(50,50),cv.FONT_HERSHEY_DUPLEX,1,0)
	#parte grafica da reta de distancia do branco
	if meio_branco < 240:
		meio_branco = 240
	cv.line(img_cv,(240,160), (int(meio_branco),160), (255,255,255),2) 
	
	if distancia_branco < 0:
		distancia_branco = 480
	texto = ('dist. branco = ' + str(int(distancia_branco)))
	cv.putText(img_cv, texto,(50,80),cv.FONT_HERSHEY_DUPLEX,1,0)

	#Coordenadas Ponto Fixo
	#print('Tamanho de Imagem: ', img.shape)
	x_medio = 480/2
	y_medio = 320/2
	cv.circle (img_cv,(int(x_medio),int(y_medio)), 10, (0,0,255), -1)
	
	#x_medium = int((x+(x+w))/2)
	pos_camera = int(meio_amarelo + ((distancia_amarelo + distancia_branco)/2))
	cv.circle (img_cv,(pos_camera,160), 10, (0,255,0), 3)
	#plt.imshow(img_cv)
	#time.sleep(0.03)
	cv.imshow("Video", img_cv)
	#video_output.write(img_cv)

##################################################################################
####################### CONTROLE DOS MOTORES
	if stop == 0:
		print("verdinho parado")
		motor_RF.ChangeDutyCycle(0)
		motor_LF.ChangeDutyCycle(0)
		motor_RB.ChangeDutyCycle(0)
		motor_LB.ChangeDutyCycle(0)
	else:   
		if abs(240 - pos_camera) < d:
			motor_RF.ChangeDutyCycle(velocidade)
			motor_LF.ChangeDutyCycle(velocidade)
			motor_RB.ChangeDutyCycle(0)
			motor_LB.ChangeDutyCycle(0)
			print("para frente")
		else:
			erro = abs(240 - pos_camera)
			i_erro += erro
			d_erro = erro - erro_anterior
			erro_anterior = erro
			controle= KP*erro + KI*i_erro + KD*d_erro
			forte = 0#velocidade+controle
			fraco = 0#velocidade-controle
			if forte > 100:
				forte = 100 
			if fraco < 0:
				fraco = 0
			if (240 - pos_camera) < -d:
				motor_RF.ChangeDutyCycle(fraco)
				motor_LF.ChangeDutyCycle(forte) #gira mais forte
				motor_RB.ChangeDutyCycle(0)
				motor_LB.ChangeDutyCycle(0)
				print(forte)
				print("para direita")  
			elif (240 - pos_camera) > d:      
				motor_RF.ChangeDutyCycle(forte) #gira mais forte
				motor_LF.ChangeDutyCycle(fraco)
				motor_RB.ChangeDutyCycle(0)
				motor_LB.ChangeDutyCycle(0)
				print("para esquerda")

	k = cv.waitKey(30) & 0xff
	if k == 27:
		break

captura.release()
cv.destroyAllWindows()
motor_RF.stop()
motor_RB.stop()
motor_LF.stop()
motor_LB.stop()
GPIO.cleanup()